In [13]:
import importlib
from ucr_benchmark_template.modeling import train_noprop

importlib.reload(train_noprop)

from ucr_benchmark_template.modeling.train_noprop import (
    load_dataset, make_model, train, predict, evaluate, save_model
)

In [30]:
import optuna
import numpy as np
import pandas as pd
import os

def objective(trial):
    try:
        datasets = [
            "ACSF1", "Adiac", "ArrowHead", "BME", "Beef", "BeetleFly", "BirdChicken", "CBF", "Car", "Chinatown",
            "ChlorineConcentration", "CinCECGTorso", "Coffee", "Computers","CricketX", "CricketY", "CricketZ", "Crop",
            "DiatomSizeReduction", "DistalPhalanxOutlineAgeGroup", "DistalPhalanxOutlineCorrect", "DistalPhalanxTW",
            "ECG200", "ECG5000", "ECGFiveDays", "EOGHorizontalSignal", "EOGVerticalSignal", "Earthquakes", "ElectricDevices",
            "EthanolLevel", "FaceAll", "FaceFour", "FacesUCR", "FiftyWords", "Fish", "FordA", "FordB", "FreezerRegularTrain",
            "FreezerSmallTrain", "GunPoint", "GunPointAgeSpan", "GunPointMaleVersusFemale", "GunPointOldVersusYoung", "Ham",
            "HandOutlines", "Haptics", "Herring", "HouseTwenty", "InlineSkate", "InsectEPGRegularTrain", "InsectEPGSmallTrain",
            "InsectWingbeatSound", "ItalyPowerDemand", "LargeKitchenAppliances", "Lightning2", "Lightning7", "Mallat", "Meat",
            "MedicalImages", "MiddlePhalanxOutlineAgeGroup", "MiddlePhalanxOutlineCorrect", "MiddlePhalanxTW",
            "MixedShapesRegularTrain", "MixedShapesSmallTrain", "MoteStrain", "NonInvasiveFetalECGThorax1",
            "NonInvasiveFetalECGThorax2", "OSULeaf", "OliveOil", "PhalangesOutlinesCorrect", "Phoneme", "PigAirwayPressure",
            "PigArtPressure", "PigCVP", "Plane", "PowerCons", "ProximalPhalanxOutlineAgeGroup", "ProximalPhalanxOutlineCorrect",
            "ProximalPhalanxTW", "RefrigerationDevices", "Rock", "ScreenType", "SemgHandGenderCh2", "SemgHandMovementCh2",
            "SemgHandSubjectCh2", "ShapeletSim", "ShapesAll", "SmallKitchenAppliances", "SmoothSubspace",
            "SonyAIBORobotSurface1", "SonyAIBORobotSurface2", "StarLightCurves", "Strawberry", "SwedishLeaf", "Symbols",
            "SyntheticControl", "ToeSegmentation1", "ToeSegmentation2", "Trace", "TwoLeadECG", "TwoPatterns", "UMD",
            "UWaveGestureLibraryAll", "UWaveGestureLibraryX", "UWaveGestureLibraryY", "UWaveGestureLibraryZ", "Wafer", "Wine",
            "WordSynonyms", "Worms", "WormsTwoClass", "Yoga"
        ]
    
        # --- Hyperparameters ---
        T = trial.suggest_int("T", 2, 10)
        emb_d = trial.suggest_categorical("embedding_dim", [0, 8, 16, 32, 64])
        eta = trial.suggest_float("eta", 0.1, 1.0, step=0.1)
        lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
        k_size = trial.suggest_categorical("k_size", [3, 5])
        dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.1)
        
        #wd = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
        
        channels = []
        fc_layers = [] 
        fc_merged = []


        
        n_blocks = trial.suggest_int("n_blocks", 2, 4)
        base_ch = trial.suggest_categorical("base_channels", [8, 16, 32, 64])
        channels = [base_ch * (2 ** i) for i in range(n_blocks)]
        
    
        n_layers = trial.suggest_int("n_layers", 1, 3)
        fc_layers = n_layers * [256]

        n_merged = trial.suggest_int("n_layers", 1, 3)
        base = trial.suggest_categorical("base_merged", [128, 256, 512])
        fc_merged = [max(base // (2 ** i), 64) for i in range(n_merged)]


        hp_str = (
            f"T={T}, emb={emb_d}, eta={eta}, lr={lr:.1e}, k={k_size}, "
            f"ch={channels}, fc={fc_layers}, merg = {fc_merged}"
            #, wd={wd:.1e}, batch={batch}, epochs={epochs}"
        )
        print(hp_str)
        
        # --- Evaluate ---
        accuracies = []
        for dataset in datasets:
            try:
                print(dataset)
                trainloader, testloader = load_dataset(dataset, batch_size = 64)
                model = make_model(dataset, emb_d, T, k_size, dropout, channels, fc_layers, fc_merged)
                model, _ = train(model, trainloader, 600, T, eta, lr, 1e-5)
                y_true, y_pred = predict(model, testloader)
                acc = evaluate(y_true, y_pred)["accuracy"]
                accuracies.append(acc)
                print(acc)
            except Exception as e:
                print(f"Error on dataset {dataset}: {e}")
                accuracies.append(np.nan)
                continue
        
        avg_acc = np.mean(accuracies)
    
        # --- Save results to one CSV with all runs ---
        csv_path = "optuna_results.csv"
    
        # Create base table if doesn't exist
        if not os.path.exists(csv_path):
            df = pd.DataFrame({"Dataset": datasets + ["AVERAGE"]})
        else:
            df = pd.read_csv(csv_path)
    
        # Make trial name
        trial_name = f"Trial_{trial.number+1}"
    
        # Add this run’s results
        run_data = accuracies + [avg_acc]
        df[trial_name] = run_data
    
        # Also add hyperparameters as a separate column name pattern
        hp_str = (
            f"T={T}, emb={emb_d}, eta={eta}, lr={lr:.1e}, k={k_size}, "
            f"ch={channels}, fc={fc_layers}, merg = {fc_merged}"
            #, wd={wd:.1e}, batch={batch}, epochs={epochs}"
        )
        print(hp_str)
        df.loc[df["Dataset"] == "AVERAGE", f"{trial_name}_params"] = hp_str
    
        df.to_csv(csv_path, index=False)
    
        return avg_acc

    except Exception as e:
        print(f"Trial {trial.number} failed with error: {e}")
        return np.nan

# --- Run Study ---
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

[I 2025-10-08 12:02:20,749] A new study created in memory with name: no-name-74528049-7838-49b5-89a4-0ba45c88193c


T=10, emb=32, eta=0.2, lr=9.9e-04, k=3, ch=[64, 128, 256], fc=[256, 256, 256], merg = [512, 256, 128]
ACSF1
0.62
Adiac
0.05115089514066496
ArrowHead
0.3942857142857143
BME
0.3333333333333333
Beef
0.2
BeetleFly
0.5
BirdChicken
0.5
CBF
0.33111111111111113
Car
0.31666666666666665
Chinatown
0.27405247813411077
ChlorineConcentration


[W 2025-10-08 12:17:10,506] Trial 0 failed with parameters: {'T': 10, 'embedding_dim': 32, 'eta': 0.2, 'lr': 0.0009930224866200486, 'k_size': 3, 'dropout': 0.1, 'n_blocks': 3, 'base_channels': 64, 'n_layers': 3, 'base_merged': 512} because of the following error: The value nan is not acceptable.
[W 2025-10-08 12:17:10,507] Trial 0 failed with value nan.


0.7924479166666667
CinCECGTorso
Error on dataset CinCECGTorso: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/CinCECGTorso.npz'
Coffee
Error on dataset Coffee: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/Coffee.npz'
Computers
Error on dataset Computers: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/Computers.npz'
CricketX
Error on dataset CricketX: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/CricketX.npz'
CricketY
Error on dataset CricketY: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/CricketY.npz'
CricketZ
Error on dataset CricketZ: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/CricketZ.npz'
Crop
Error on dataset Crop: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/Crop.np

[W 2025-10-08 12:17:10,953] Trial 1 failed with parameters: {'T': 4, 'embedding_dim': 8, 'eta': 0.9, 'lr': 0.0005330264376583687, 'k_size': 5, 'dropout': 0.1, 'n_blocks': 2, 'base_channels': 8, 'n_layers': 3, 'base_merged': 256} because of the following error: The value nan is not acceptable.
[W 2025-10-08 12:17:10,954] Trial 1 failed with value nan.


...

0.2
BeetleFly
Error on dataset BeetleFly: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/BeetleFly.npz'
BirdChicken
Error on dataset BirdChicken: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/BirdChicken.npz'
CBF
Error on dataset CBF: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/CBF.npz'
Car
Error on dataset Car: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/Car.npz'
Chinatown
Error on dataset Chinatown: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/Chinatown.npz'
ChlorineConcentration
Error on dataset ChlorineConcentration: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/ChlorineConcentration.npz'
CinCECGTorso
Error on dataset CinCECGTorso: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/proce

[W 2025-10-08 12:17:12,638] Trial 3 failed with parameters: {'T': 9, 'embedding_dim': 8, 'eta': 0.4, 'lr': 0.0027292870656349037, 'k_size': 5, 'dropout': 0.2, 'n_blocks': 3, 'base_channels': 32, 'n_layers': 2, 'base_merged': 512} because of the following error: The value nan is not acceptable.
[W 2025-10-08 12:17:12,639] Trial 3 failed with value nan.


0.2
BeetleFly
Error on dataset BeetleFly: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/BeetleFly.npz'
BirdChicken
Error on dataset BirdChicken: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/BirdChicken.npz'
CBF
Error on dataset CBF: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/CBF.npz'
Car
Error on dataset Car: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/Car.npz'
Chinatown
Error on dataset Chinatown: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/Chinatown.npz'
ChlorineConcentration
Error on dataset ChlorineConcentration: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/ChlorineConcentration.npz'
CinCECGTorso
Error on dataset CinCECGTorso: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/proce

[W 2025-10-08 12:17:13,346] Trial 4 failed with parameters: {'T': 5, 'embedding_dim': 32, 'eta': 0.6, 'lr': 0.002981460025146855, 'k_size': 3, 'dropout': 0.30000000000000004, 'n_blocks': 3, 'base_channels': 64, 'n_layers': 1, 'base_merged': 512} because of the following error: The value nan is not acceptable.
[W 2025-10-08 12:17:13,346] Trial 4 failed with value nan.


0.2
BeetleFly
Error on dataset BeetleFly: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/BeetleFly.npz'
BirdChicken
Error on dataset BirdChicken: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/BirdChicken.npz'
CBF
Error on dataset CBF: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/CBF.npz'
Car
Error on dataset Car: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/Car.npz'
Chinatown
Error on dataset Chinatown: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/Chinatown.npz'
ChlorineConcentration
Error on dataset ChlorineConcentration: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/processed/mlp/ChlorineConcentration.npz'
CinCECGTorso
Error on dataset CinCECGTorso: [Errno 2] No such file or directory: '/home/jovyan/UCRBenchmarkTemplate/data/proce